# RDD

an RDD is the fundamental data structure of Apache Spark. It provised low level data abstraction. It's a fault-tolerant, distributed collection of elements that can be operated on in parallel.

**Key Characteristics:**

- Immutable
- Lazy evaluation
- Fault tolerant (via lineage info)
- Partitioned across cluster nodes
- Can be cached in memory

### SparkContext and SparkConf


SparkContext is the foundational entry point for any Apache Spark application. It serves as the primary connection to a Spark cluster and is responsible for managing resources and orchestrating distributed computations.

**Key Aspects:**

- Cluster Connection: It establishes and maintains the connection between your Spark application (the driver program) and the Spark cluster
- Resource Management:SparkContext interacts with the cluster manager to acquire and manage the computational resources (executors, memory, CPU cores) needed for your application's tasks.
- RDD Creation: SparkContext was the primary way to create Resilient Distributed Datasets (RDDs), which are the fundamental low-level data abstraction in Spark.

#### `SparkConf`

SparkConf is a configuration class used to set various Spark properties and parameters for a Spark application. It allows you to define settings such as the application name, the master URL (where the Spark cluster is located), memory allocations for executors, number of CPU cores, and other specific configurations that control Spark's behavior. A SparkConf object is typically used to initialize a SparkContext or SparkSession, providing the necessary environment details for the application to run on the cluster.

**Common settings:**

- setMaster("local[*]") – Use local mode with all cores
- setAppName("RDDExample") – Application name

### transformations

Transformations create a new RDD from an existing one. They are lazy – not executed until an action is triggered. They are characterized by their lazy evaluation, meaning they do not immediately execute when called. Instead, Spark builds a Directed Acyclic Graph (DAG) of the computations, which represents the lineage of transformations. 

| Transformation  | Description                                          |
| --------------- | ---------------------------------------------------- |
| `map(func)`     | Returns a new RDD by applying `func` to each element |
| `filter(func)`  | Filters elements for which `func` returns true       |
| `flatMap(func)` | Like map but flattens the result                     |
| `distinct()`    | Removes duplicates                                   |
| `union(rdd)`    | Combines two RDDs                                    |
| `groupByKey()`  | Groups values with same key                          |
| `reduceByKey()` | Aggregates values with same key using a function     |
| `sortBy(func)`  | Sorts RDD by computed key                            |


### actions

Actions trigger the execution of the entire DAG of transformations and return a result to the driver program or write data to an external storage system. They are eagerly evaluated, meaning they execute immediately when called. 

| Action             | Description                            |
| ------------------ | -------------------------------------- |
| `collect()`        | Returns all elements to driver         |
| `count()`          | Returns number of elements             |
| `first()`          | Returns first element                  |
| `take(n)`          | Returns first `n` elements             |
| `reduce(func)`     | Reduces elements using binary operator |
| `saveAsTextFile()` | Writes RDD to text files               |



reference - [spark rdd docs](https://spark.apache.org/docs/latest/rdd-programming-guide.html)


### Shared Variables

When you pass a function (like in map or reduce) to Spark, that function runs on different machines in the cluster — not on your local driver.

**By default:**

- Spark makes separate copies of any variable you use inside those functions.
- So, if a task changes a variable on the executor, that change will not reflect back in your driver program.

This is done to keep things fast and distributed — but it also means you can not just update normal variables across tasks.

**challenges:**

You want to count how many rows have Amount > 500 using this code:

```py
count = 0

def increment_count(x):
    count += 1 

records.filter(lambda x: int(x[2]) > 500).foreach(increment_count)
print(count)

```

This will not work because each machine updates its own copy of count not the original one in the driver.

To solve such issues, spark provides two types of shared variables:

1. **Broadcast Variables:** a read-only variable that can be cached on each machine (executor). Used to efficiently share large data (like lookup tables) with all tasks without copying it multiple times.
2. **Accumulators:** variables used to safely implement counters or sums across mulitple worker nodes. You can only add to them (not read or subtract inside tasks). The final value is only accessible on the driver after an action is executed.

Spark shared variables are a mechanism for efficiently distributing read-only data or aggregating results across executors in a Spark application.

# Activity


1. Find all product IDs where the amount is greater than 900.
2. Find all transactions that belong to the “Furniture” category.
3. Count how many transactions belong to the “Electronics” category.
4. Find average amount for each category.
5. Find the highest amount and the corresponding product ID.
6. Find the total number of unique categories.
7. For each category, find the product ID with the highest sale.
8. Count how many products were sold for less than 300.
9. Sort the transactions by amount in descending order.
10. Broadcast Variable: Category Discounts
```{"Electronics": 0.10, "Furniture": 0.15, "Clothing": 0.05, "Books": 0.20}```
11. Accumulator: Count Transactions Below 300
12. Filter and Save Results